In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

/Users/jinny/.pyenv/versions/3.14.2/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/Users/jinny/.pyenv/versions/3.14.2/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# [4] 하위 Agent의 Tool

In [3]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str], #email
    location: str = '', # 위치가 없을 경우, 빈 문자열을 기본값으로
):
    '''캘린더 이벤트 생성'''
    return f'이벤트 생성 완료. {title} - {start_time} ~ {end_time}. '


@tool
def get_available_time_slot(
    attendees: list[str], #email,
    date: str,
    duration_minutes: int
):
    '''참가자들이 특정 날짜에 참여 가능한 시간 확인'''

    return ['09:00', '14:00', '16:00']

In [4]:
@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    attendees: list[str], #email

):
    '''이메일 발송'''
    return f'이메일 발송 완료. {to} - {subject}'

# [3] 하위 에이전트

In [5]:
from langchain.agents import create_agent
from datetime import datetime

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    f'"NOW: {datetime.now()}"'
)

calendar_agent = create_agent(
    model,
    # 이 부분에 calendar_agent가 사용할 tool 목록을 작성해야함
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [6]:
from langchain_community.utilities import SQLDatabase
import os
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent
from datetime import datetime

DB_URI = os.environ.get('DB_URI')
db = SQLDatabase.from_uri(DB_URI)

toolkit = SQLDatabaseToolkit(db=db, llm=model)
dialect = db.dialect

SQL_AGENT_PROMPT = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. 

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.

추가 지침:
1. 사용자가 말한 팀 이름이 DB에 정확히 없을 수 있습니다. 
   반드시 'LIKE %키워드%'를 사용하거나, 팀 목록 테이블을 먼저 전체 조회하여 가장 유사한 팀을 찾아내세요.
2. 예: '인사 팀' 요청 시 -> '인사', 'HR', '인사관리' 등을 모두 염두에 두어야 합니다.
3. 이메일 주소를 찾을 때는 직원(employees) 테이블과 팀(teams/departments) 테이블을 적절히 조인(JOIN)하세요.
"""


sql_agent = create_agent(
    model,
    # 이 부분에 agent가 사용할 tool 목록을 작성해야함
    tools=toolkit.get_tools(),
    system_prompt=SQL_AGENT_PROMPT
)

In [7]:
from langchain.agents import create_agent

EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    # 이 부분에 agent가 사용할 tool 목록을 작성해야함
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

# [2] 상위 에이전트 Supervisor Agent의 Tool

In [8]:
from langchain.tools import tool

@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    # request 부분은 agent가 작성해서 보냄.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

@tool
def get_employees_emails(request: str) -> str:
    """SQL 쿼리를 이용해 필요한 이메일 주소를 받아옴. 
    부서명이나 팀 이름을 입력하면 해당 소속 직원들의 이메일 리스트를 반환함.
    """
    
    full_request = f"데이터베이스에서 '{request}' 소속 직원들의 이메일 주소를 모두 찾아줘. 결과는 이메일 주소만 나열해줘."

    result = sql_agent.invoke({
        "messages": [{"role": "user", "content": full_request}]
    })
    
    return result["messages"][-1].content 
    
@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

# [1] 상위 에이전트 Supervisor Agent

In [9]:
SUPERVISOR_PROMPT = '''너는 매우 유능한 개인 비서야.
너는 캘린더 이벤트를 조정하고, 이메일을 보낼 수 있어. SQL을 이용해서 직원 정보에서 이메일 주소도 확인할 수 있어.
사용자의 요청을 분석하여, 적절한 도구를 사용하고, 결과를 종합해야 해.
요청이 여러가지 액션을 취해야 하면, 순서를 잘 짜서 각종 도구들을 여러번 호출해.
'''

supervisor_agent = create_agent(
    model,
    # 이 부분에 supervisor_agent가 사용할 tool 목록을 작성해야함
    tools=[schedule_event, get_employees_emails, manage_email ], 
    system_prompt=SUPERVISOR_PROMPT,
)

In [10]:
# supervisor agent에 입력할 사용자 input 메시지
query = '다음주 화요일 오후2시부터 1시간동안 인사 팀 미팅 잡고, 메일 다 보내놔.'

for step in supervisor_agent.stream(
    {'messages': [{'role': 'user', 'content': query}]}
):
    for update in step.values():
        for message in update.get('messages', []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_FCMDdqIbWcD68k6kYBXogMR9)
 Call ID: call_FCMDdqIbWcD68k6kYBXogMR9
  Args:
    request: 인사 팀 미팅 다음주 화요일 오후 2시부터 1시간
================================= Tool Message =================================
Name: schedule_event

인사 팀 미팅을 다음주 화요일인 2026년 3월 3일 오후 2시부터 1시간 동안 일정에 추가했습니다.
================================== Ai Message ==================================
Tool Calls:
  get_employees_emails (call_6Vj6pZNM6khpEOauVQQhP8Ja)
 Call ID: call_6Vj6pZNM6khpEOauVQQhP8Ja
  Args:
    request: 인사 팀
================================= Tool Message =================================
Name: get_employees_emails

'인사 팀'은 데이터베이스 내에서 'HR' 팀으로 확인되었습니다. 해당 팀에 소속된 직원들의 이메일 주소는 다음과 같습니다:
hr1@company.com
hr2@company.com
hr3@company.com
hr4@company.com
hr5@company.com
================================== Ai Message ==================================
Tool Calls:
  manage_email (call_oVLgyGBTXXzFd2yVM11YlG